In [1]:
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from urllib.parse import urlparse
import pickle
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
fn = "df_lem_ftr1.pk"
df = pd.read_pickle(fn)
df.sample(5)

,labels,text,text_tokens,text_cleaned
6854,1,Obamacare medical device tax has led to the lo...,"(Obamacare, medical, device, tax, has, led, to...",obamacare medical device tax have lead to loss...
2338,0,"In 2008, then-Sen. Barack Obama missed 60 or 7...","(In, 2008, ,, then, -, Sen., Barack, Obama, mi...",in then Sen. Barack Obama miss percent of his ...
4636,1,There are 3 million jobs available in America ...,"(There, are, 3, million, jobs, available, in, ...",there be job available in America that be fill...
2153,0,"While in the Illinois Senate, Barack Obama pas...","(While, in, the, Illinois, Senate, ,, Barack, ...",in Illinois Senate Barack Obama pass tax cut f...
8431,1,Says Republican U.S. Rep. Bob Dold continues t...,"(Says, Republican, U.S., Rep., Bob, Dold, cont...",say republican U.S. Rep. Bob Dold continue sid...


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.naive_bayes import MultinomialNB

Y = df["labels"]
X = df["text_cleaned"]

pipeline = Pipeline(
    [
        ("CV", CountVectorizer()), 
        ("mnb", MultinomialNB(class_prior=None))
    ]
)

param_grid = {
    
    
    "CV__ngram_range": [(1,1),(1,2)],
    "CV__max_df": [0.95,0.90,0.85,0.80,0.75,0.70,0.65,0.60,0.55,0.50, 0.45, 0.40],
    "CV__min_df": [0.0001, 0.00025, 0.0005, 0.001, 0.0025, 0.005],
    "CV__binary": [True,False],
    "mnb__alpha": [1, 4, 16, 64, 128, 256, 512],
    "mnb__fit_prior": [False]
    
    
    
}

SCORING = "f1"


grid = GridSearchCV(
    pipeline,
    n_jobs=5,
    param_grid=param_grid,
    verbose=1,
    # réentrainer le meilleur modèle sur toutes les données
    refit=True,
    scoring=SCORING,
)





grid.fit(X, Y)

Fitting 5 folds for each of 2016 candidates, totalling 10080 fits


GridSearchCV(estimator=Pipeline(steps=[('CV', CountVectorizer()),
                                       ('mnb', MultinomialNB())]),
             n_jobs=5,
             param_grid={'CV__binary': [True, False],
                         'CV__max_df': [0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65,
                                        0.6, 0.55, 0.5, 0.45, 0.4],
                         'CV__min_df': [0.0001, 0.00025, 0.0005, 0.001, 0.0025,
                                        0.005],
                         'CV__ngram_range': [(1, 1), (1, 2)],
                         'mnb__alpha': [1, 4, 16, 64, 128, 256, 512],
                         'mnb__fit_prior': [False]},
             scoring='f1', verbose=1)

In [4]:
grid.best_estimator_

Pipeline(steps=[('CV',
                 CountVectorizer(binary=True, max_df=0.95, min_df=0.0025,
                                 ngram_range=(1, 2))),
                ('mnb', MultinomialNB(alpha=512, fit_prior=False))])

In [5]:
import joblib
joblib.dump(grid, '4_ml_nb_lf1_cv_best.pkl')

['4_ml_nb_lf1_cv_best.pkl']

In [6]:
grid.best_score_

0.7603513477735412

In [7]:
len(grid.best_estimator_['CV'].vocabulary_)

1343